In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False: # if True: turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import random


from utils.cityscapesSequence import CitySequence, labels
import utils.dataUtils as dataUtils
from modelclass import DeeplabV3

if gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 8))])

print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.1


In [2]:
DATA_DIR = os.path.join("..", "cityscapes", "allData")

batchSize = 32
imageSize = 256
lrs = [5e-4, 1e-4, 5e-5, 1e-5, 5e-6]
alphas = [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]


##### Dataset


In [3]:
def getData(path="", type="gtFine", extra=False, nSplits=5, seed=0):
    # Load data
    xTrainVal, yTrainVal, xTest, yTest = dataUtils.getXY(path=path, extra=extra, type=type)
    print("TrainVal: ", len(xTrainVal), "Test: ", len(yTest))

    # Split the data in train and validation folds
    return dataUtils.splitTrainValFolds(xTrainVal, yTrainVal, nSplits=nSplits, seed=seed)


def createDatasetObject(trainValFolds, imageSize=imageSize, batchSize=1, remap="binary"):
    for i, (xTrain, yTrain, xVal, yVal) in enumerate(trainValFolds):
        print("Fold", i, "Train: ", len(xTrain), "Val: ", len(xVal))

        dataUtils.dataShuffle(xData=xTrain, yData=yTrain)

        # Create the dataset
        trainDataset = CitySequence(xTrain, yTrain, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=True, verticalFlip=True, brightness=0.1)
        valDataset = CitySequence(xVal, yVal, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=False, verticalFlip=False, brightness=0)

        yield trainDataset, valDataset, trainDataset.nClasses


kf = getData(path=DATA_DIR, type="gtFine", extra=False, nSplits=5, seed=13)
data = createDatasetObject(kf, imageSize=imageSize, batchSize=batchSize, remap="binary")

Getting data from: ..\cityscapes\allData
Getting train data from: ..\cityscapes\allData\leftImg8bit\train\*\*_leftImg8bit.png
Getting val data from: ..\cityscapes\allData\leftImg8bit\val\*\*_leftImg8bit.png
Getting test data from: ..\cityscapes\allData\leftImg8bit\test\*\*_leftImg8bit.png
TrainVal:  3475 Test:  1525


##### Train

In [4]:
losses = {}

for deeplayer in [6, 7, 8, 9, 10, 11, 12]:
    mobileLayers={"shallowLayer": "block_2_project_BN", "deepLayer": "block_" + str(deeplayer) + "_project_BN"}
    
    for lr in lrs:
        for alpha in alphas:
            for i, (trainDataset, valDataset, nClasses) in enumerate(data):
                if alpha > 1. and deeplayer > 10:
                    continue
                
                print("Fold:", i, "learning rate:", lr, "alpha:", alpha)
                
                MODEL_NAME = "deeplabv3plus_fold_" + str(i) + "_lr_" + str(lr) + "_alpha_" + str(alpha) + "_binary_" + str(imageSize) + "x" + str(imageSize) + "_deeplayer_" + str(deeplayer)

                model = DeeplabV3(imageSize=imageSize, nClasses=nClasses, alpha=alpha, modelName=MODEL_NAME, mobileLayers=mobileLayers)
                
                model.run(trainDataset, valDataset, epochs=1000, learningRate=lr, batchSize=batchSize, monitor="val_loss", log=True)

                # Evaluate the model
                losses[MODEL_NAME] = model.evaluate(valDataset)
                print("losses", losses)

Fold 0 Train:  2780 Val:  695
Fold: 0 learning rate: 0.0005 alpha: 0.35
Epoch 1/1000
348/348 [==============================] - ETA: 0s - loss: 52.1352 - diceAccuracy: 0.8846 - jaccardDistance: 1.9902
Epoch 1: val_loss improved from inf to 14.46461, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6\assets


348/348 [==============================] - 405s 1s/step - loss: 52.1352 - diceAccuracy: 0.8846 - jaccardDistance: 1.9902 - val_loss: 14.4646 - val_diceAccuracy: 0.4885 - val_jaccardDistance: 6.7660 - lr: 5.0000e-04
Epoch 2/1000
348/348 [==============================] - ETA: 0s - loss: 8.8318 - diceAccuracy: 0.9221 - jaccardDistance: 1.4111
Epoch 2: val_loss improved from 14.46461 to 5.00204, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6\assets


...

348/348 [==============================] - 350s 1s/step - loss: 0.0351 - diceAccuracy: 0.9485 - jaccardDistance: 0.9671 - val_loss: 0.0359 - val_diceAccuracy: 0.9472 - val_jaccardDistance: 0.9865 - lr: 1.9531e-06
Epoch 360/1000
348/348 [==============================] - ETA: 0s - loss: 0.0354 - diceAccuracy: 0.9480 - jaccardDistance: 0.9756
Epoch 360: val_loss did not improve from 0.03585
348/348 [==============================] - 333s 958ms/step - loss: 0.0354 - diceAccuracy: 0.9480 - jaccardDistance: 0.9756 - val_loss: 0.0370 - val_diceAccuracy: 0.9455 - val_jaccardDistance: 1.0151 - lr: 1.9531e-06
Epoch 361/1000
348/348 [==============================] - ETA: 0s - loss: 0.0353 - diceAccuracy: 0.9481 - jaccardDistance: 0.9740
Epoch 361: val_loss did not improve from 0.03585
348/348 [==============================] - 331s 949ms/step - loss: 0.0353 - diceAccuracy: 0.9481 - jaccardDistance: 0.9740 - val_loss: 0.0367 - val_diceAccuracy: 0.9459 - val_jaccardDistance: 1.0071 - lr: 1.9531e-

INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_0.35_binary_256x256_deeplayer_6\assets


348/348 [==============================] - 348s 999ms/step - loss: 0.0356 - diceAccuracy: 0.9477 - jaccardDistance: 0.9814 - val_loss: 0.0357 - val_diceAccuracy: 0.9474 - val_jaccardDistance: 0.9808 - lr: 1.9531e-06
Epoch 376/1000
348/348 [==============================] - ETA: 0s - loss: 0.0351 - diceAccuracy: 0.9484 - jaccardDistance: 0.9689
Epoch 376: val_loss did not improve from 0.03572
348/348 [==============================] - 331s 951ms/step - loss: 0.0351 - diceAccuracy: 0.9484 - jaccardDistance: 0.9689 - val_loss: 0.0366 - val_diceAccuracy: 0.9460 - val_jaccardDistance: 1.0072 - lr: 1.9531e-06
Epoch 377/1000
348/348 [==============================] - ETA: 0s - loss: 0.0352 - diceAccuracy: 0.9483 - jaccardDistance: 0.9710
Epoch 377: val_loss did not improve from 0.03572
348/348 [==============================] - 333s 958ms/step - loss: 0.0352 - diceAccuracy: 0.9483 - jaccardDistance: 0.9710 - val_loss: 0.0364 - val_diceAccuracy: 0.9463 - val_jaccardDistance: 1.0001 - lr: 1.953

KeyboardInterrupt: 

In [ ]:
losses

{}

##### TFLITE Converter


In [ ]:
from utils.imageUtils import representativeDatasetGen

representativeData = representativeDatasetGen(path="../cityscapes/alldata")

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData, supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS])

MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData)

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)

model.load_weights(MODEL_PATH)

